In [228]:
import sqlite3
from sqlite3 import Error
from tkinter import Tk, filedialog
import pandas as pd
import numpy as np
import random

# file selection packages 
import traitlets
from IPython.display import display
from ipywidgets import widgets
from tkinter import Tk, filedialog

In [229]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self, *args, **kwargs):
        """Initialize the SelectFilesButton class."""
        super(SelectFilesButton, self).__init__(*args, **kwargs)
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.
        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True)

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"
file = SelectFilesButton()
file

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [230]:
# Load in file as a DataFrame
new_df = pd.read_csv(file.files[0])

In [231]:
# drop the duplicate sentences
new_df.drop_duplicates(subset='Input.text', keep='first', inplace=True)

# mix the df by a random sequence of numbers
for i in range(5):
    random_list = random.sample(range(len(new_df['Input.text'])), len(new_df['Input.text']))
    new_df['scramble_seq'] = random_list # add the list to the dataframe
    new_df = new_df.sort_values(by='scramble_seq')
    

# drop the random number sequence column that scrambles
new_df.drop(columns=['scramble_seq'], inplace=True) # drop the column used to scramble data
new_df.reset_index(drop=True, inplace=True) # reset the index

In [232]:
# Ask for user input to see what they want to chnage
# IMPORTANT NOTE: If filtering by confidence value must type ALL decimal places with '%' (e.g. '100.00%')

for thing in turk_file.columns:
    filt = input("Do you want to filter by " + str(thing) + " ? (y/n)")
    if filt == 'y':
        print('this works')
        by = input('type what you want ' + str(thing) + ' to be')
        if thing == 'is_stressor' or thing == 'is_covid':
            print("how'd you get here")
            by = int(by)
        new_df = new_df[new_df[thing] == by]
    more = input('Do you want to filter by another aspect? (y/n)')
    if more == 'n':
        break

 

Do you want to filter by sID ? (y/n)
Do you want to filter by another aspect? (y/n)
Do you want to filter by Input.text ? (y/n)
Do you want to filter by another aspect? (y/n)
Do you want to filter by is_stressor ? (y/n)y
this works
type what you want is_stressor to be1
how'd you get here
Do you want to filter by another aspect? (y/n)
Do you want to filter by is_stressor_conf ? (y/n)y
this works
type what you want is_stressor_conf to be100.00%
Do you want to filter by another aspect? (y/n)n


In [233]:
# Creating the test set list

test_set = [] # create big list that has all the sentences that are going to make the test set across all labels
labels = new_df['top_label'].unique().tolist()
for label in labels:
    sentence_list = new_df[new_df['top_label'] == label]['Input.text'].tolist()
    count = len(sentence_list)
    test_amount = count * 0.2
    added = 0
    while added <= test_amount:
        index = random.randint(0, count - 1)
        test_set.append(sentence_list[index])       
        del sentence_list[index]
        added += 1
        count -= 1

In [234]:
# Create a list of 0's
set_list = []
sentences = new_df['Input.text'].tolist()
for i in range(len(new_df['top_label'])):
    set_list.append(0)

# put a 1 in the row where a testing sentence is 
for sentence in test_set:
    index = sentences.index(sentence)
    set_list[index] = 1

# add the testing and training list to the dataframe 
new_df['set'] = set_list

In [235]:
# create a dictionary that holds a value for each label
key_dict = {}
for i in range(len(labels)):
    key_dict[labels[i]] = i

# create a list that is going to represent each label as the value from the dictionary    
Class = []
for label in new_df['top_label']:
    Class.append(key_dict[label])

# add the column to the dataframe    
new_df['class'] = Class 

In [236]:
# key for the 'class' column
print(key_dict)

{'School': 0, 'Work': 1, 'Family Issues': 2, 'Health, Fatigue, or Physical Pain': 3, 'Other': 4, 'Social Relationships': 5, 'Emotional Turmoil': 6, 'Everyday Decision Making': 7, 'Financial Problem': 8}


In [237]:
new_df
# should I reset the index again here? 
# Is this what we want the form to look like?

,sID,Input.text,is_stressor,is_stressor_conf,is_covid,is_covid_conf,top_label,second_label,avg_severity,median_severity,...,Emotional Turmoil,School,Family Issues,Social Relationships,Work,"Health, Fatigue, or Physical Pain",Source,is_seed,set,class
0,6605,"i have to worry about college, and school sta...",1,100.00%,0,100.00%,School,,3.6,4,...,0.0,1.0,0.0,0.0,0.0,0.0,Inquire,0,0,0
1,1772,"my boss has been crazy lately, threatened to l...",1,100.00%,0,100.00%,Work,,5.6,5,...,0.0,0.0,0.0,0.0,1.0,0.0,mTurk_synthetic,0,0,1
2,6316,I'm scared of what my family will say.,1,100.00%,0,100.00%,Family Issues,Emotional Turmoil,4.2,4,...,0.3,0.0,0.7,0.0,0.0,0.0,Inquire,0,0,2
3,3269,"I'm tired, and not feeling very well.",1,100.00%,0,100.00%,"Health, Fatigue, or Physical Pain",,3.4,3,...,0.0,0.0,0.0,0.0,0.0,1.0,Inquire,0,0,3
4,4973,This class is the worst part of the whole day.,1,100.00%,0,100.00%,School,Emotional Turmoil,3.2,3,...,0.1,0.8,0.0,0.0,0.0,0.0,Inquire,0,1,0
5,805,I had a miscommunication with my boss.,1,100.00%,0,100.00%,Work,Everyday Decision Making,3.2,3,...,0.0,0.0,0.0,0.0,0.9,0.0,mTurk_synthetic_covid,0,1,1
6,3312,"I'm physically exhausted, and feeling this way...",1,100.00%,0,100.00%,"Health, Fatigue, or Physical Pain",Emotional Turmoil,3.4,4,...,0.3,0.0,0.0,0.0,0.0,0.7,Inquire,0,0,3
8,1412,issues we have at work about data,1,100.00%,0,100.00%,Work,,2.8,2,...,0.0,0.0,0.0,0.0,1.0,0.0,mTurk_synthetic,0,0,1
10,1977,my job and the lack of communication in that p...,1,100.00%,0,100.00%,Work,,3.0,3,...,0.0,0.0,0.0,0.0,1.0,0.0,mTurk_synthetic,0,0,1
11,3893,lately I've been getting really sad when I thi...,1,100.00%,0,100.00%,Family Issues,Emotional Turmoil,5.0,5,...,0.4,0.0,0.6,0.0,0.0,0.0,InquireComposited,0,0,2
